In [5]:
import json
from pathlib import Path
import numpy as np
import csv

from analysis.coverege_per_niche import count_points_per_niche_dtlz2, analyze_niche_distribution
from utils.generate_points import generate_reference_points

# --- CONFIG ---
EXPERIMENT = "experiment7"
RESULTS_DIR = Path(EXPERIMENT)   # deve rodar dentro de "results/"
OUTPUT_DIR = Path("reanalyzed") / EXPERIMENT
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Raios que você quer testar
RADII = [0.15, 0.20, 0.25, 0.3, 0.35]

# --- FUNÇÕES ---

def reanalyze_coverage_for_radius(pareto_front, ref_pts, radius):
    ptin, ptout = count_points_per_niche_dtlz2(pareto_front, ref_pts, radius)
    niche_metrics = analyze_niche_distribution(ptin, ptout)
    return {
        "radius": radius,
        "points_in": ptin.tolist(),
        "points_out": ptout,
        **niche_metrics
    }

def reanalyze_experiment(results_dir: Path, output_dir: Path, radii: list[float]):
    all_metrics = {}

    for file in results_dir.glob("run_*_*.json"):
        with open(file, "r") as f:
            data = json.load(f)

        impl = data["implementation"]
        pareto_front = np.array(data["pareto_front"], dtype=float)

        # parâmetros originais do experimento
        num_obj = data.get("num_obj") or None
        divisions = data.get("divisions") or None

        if num_obj is None or divisions is None:
            # tenta pegar do summary se não estiver em cada arquivo
            # fallback: usar valores globais do experimento
            summary_file = results_dir / "summary.json"
            if summary_file.exists():
                with open(summary_file, "r") as sf:
                    summary = json.load(sf)
                params = summary.get("parameters", {})
                num_obj = params.get("num_obj", 2)
                divisions = params.get("divisions", 10)
            else:
                num_obj, divisions = 2, 10  # defaults

        ref_pts = generate_reference_points(num_obj, divisions)

        all_metrics.setdefault(impl, {})

        for r in radii:
            res = reanalyze_coverage_for_radius(pareto_front, ref_pts, r)
            all_metrics[impl].setdefault(r, []).append(res)

    # agregação
    aggregated = {}
    for impl, rad_dict in all_metrics.items():
        aggregated[impl] = {}
        for r, metrics_list in rad_dict.items():
            keys = [k for k in metrics_list[0].keys() if k not in ("radius", "points_in", "points_out")]
            mean_metrics = {}
            for k in keys:
                vals = [m[k] for m in metrics_list]
                mean_metrics[k] = float(np.mean(vals))
            aggregated[impl][r] = mean_metrics

    # salvar em JSON
    with open(output_dir / "reanalyzed_coverage.json", "w") as f:
        json.dump({
            "radii": radii,
            "aggregated": aggregated
        }, f, indent=2)

    # salvar em CSV
    csv_file = output_dir / "reanalyzed_coverage.csv"
    with open(csv_file, "w", newline="") as fcsv:
        fieldnames = ["implementation", "radius"] + list(next(iter(next(iter(aggregated.values())).values())).keys())
        writer = csv.DictWriter(fcsv, fieldnames=fieldnames)
        writer.writeheader()
        for impl, rad_dict in aggregated.items():
            for r, metric_dict in rad_dict.items():
                row = {"implementation": impl, "radius": r}
                row.update(metric_dict)
                writer.writerow(row)

    print(f"Reanálise concluída. Resultados salvos em {output_dir}")

# --- EXECUÇÃO ---
reanalyze_experiment(RESULTS_DIR, OUTPUT_DIR, RADII)


Reanálise concluída. Resultados salvos em reanalyzed\experiment7


In [9]:
import json
from pathlib import Path
import numpy as np

from analysis.indicators import hypervolume

# --- CONFIG ---
EXPERIMENTS = [f"experiment{i}" for i in range(1, 8)]

# pontos de referência (um por experimento 1..7, na mesma ordem)
REFERENCE_POINTS = [
    [5]*2, # exemplo para experiment1
    [5]*3, # experiment2
    [5]*4, # experiment3
    [5]*4, # experiment4 (se 3 objetivos)
    [5]*5, # experiment5
    [5]*6, # experiment6
    [5]*6  # experiment7
]

OUTPUT_DIR = Path("reanalyzed_hv")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


def reanalyze_hv_experiment(results_dir: Path, ref_point: list[float]):
    """Recalcula HV médio para cada implementação em um experimento"""
    stats = {}

    for file in results_dir.glob("run_*_*.json"):
        with open(file, "r") as f:
            data = json.load(f)

        impl = data["implementation"]
        pareto_front = np.array(data["pareto_front"], dtype=float)

        hv = hypervolume(pareto_front, ref_point)

        stats.setdefault(impl, []).append(hv)

    # agregação: média por implementação
    aggregated = {}
    for impl, values in stats.items():
        aggregated[impl] = {
            "mean_hv": float(np.mean(values)),
            "std_hv": float(np.std(values)),
            "num_runs": len(values)
        }

    return aggregated


def reanalyze_all_experiments(base_dir: Path, experiments: list[str], ref_points: list[list[float]]):
    summary = {
        "parameters": {
            "experiments": experiments,
            "reference_points": ref_points
        },
        "results": {}
    }

    for exp, ref in zip(experiments, ref_points):
        results_dir = base_dir / exp
        if not results_dir.exists():
            print(f"⚠ Diretório {results_dir} não encontrado, ignorando")
            continue

        print(f"🔄 Reanalisando {exp} com referência {ref}...")
        aggregated = reanalyze_hv_experiment(results_dir, ref)
        summary["results"][exp] = aggregated

    # salvar em JSON
    out_file = OUTPUT_DIR / "reanalyzed_hv_summary.json"
    with open(out_file, "w") as f:
        json.dump(summary, f, indent=2)

    print(f"✅ Reanálise concluída. Resultados salvos em {out_file}")


# --- EXECUÇÃO ---
reanalyze_all_experiments(Path('.'), EXPERIMENTS, REFERENCE_POINTS)


🔄 Reanalisando experiment1 com referência [5, 5]...
🔄 Reanalisando experiment2 com referência [5, 5, 5]...
🔄 Reanalisando experiment3 com referência [5, 5, 5, 5]...
🔄 Reanalisando experiment4 com referência [5, 5, 5, 5]...
🔄 Reanalisando experiment5 com referência [5, 5, 5, 5, 5]...
🔄 Reanalisando experiment6 com referência [5, 5, 5, 5, 5, 5]...
🔄 Reanalisando experiment7 com referência [5, 5, 5, 5, 5, 5]...
✅ Reanálise concluída. Resultados salvos em reanalyzed_hv\reanalyzed_hv_summary.json
